In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark=SparkSession.builder\
.appName("Struct Type")\
.getOrCreate()

In [2]:
raw_data = [
    ("U001","Abhishek",28,"Hyderabad",50000),
    ("U002","Neha",32,"Delhi",62000),
    ("U003","Ravi",25,"Bangalore",45000),
    ("U004","Pooja",29,"Mumbai",58000)
]

In [3]:
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType, LongType
)

In [4]:
user_schema = StructType([
    StructField("user_id",StringType(),False),
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True),
    StructField("city",StringType(),True),
    StructField("salary",LongType(),True)
])

In [5]:
df_users = spark.createDataFrame(raw_data, schema=user_schema)
df_users.printSchema()
df_users.show()

root
 |-- user_id: string (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- salary: long (nullable = true)

+-------+--------+---+---------+------+
|user_id|    name|age|     city|salary|
+-------+--------+---+---------+------+
|   U001|Abhishek| 28|Hyderabad| 50000|
|   U002|    Neha| 32|    Delhi| 62000|
|   U003|    Ravi| 25|Bangalore| 45000|
|   U004|   Pooja| 29|   Mumbai| 58000|
+-------+--------+---+---------+------+



In [6]:
raw_data2 = [
    ("U005","Amit","Thirty","Chennai",40000)
]

In [7]:
df_users2 = spark.createDataFrame(raw_data2, schema=user_schema)
df_users2.show()

PySparkTypeError: [FIELD_DATA_TYPE_UNACCEPTABLE_WITH_NAME] field age: IntegerType() can not accept object 'Thirty' in type <class 'str'>.

In [8]:
from pyspark.sql.types import ArrayType

In [9]:
interest_data = [
    ("U001",["AI","ML","Cloud"]),
    ("U002",["Testing","Automation"]),
    ("U003",["Data Engineering","Spark","Kafka"]),
    ("U004",["UI/UX"])
]

In [10]:
interest_schema=StructType([
    StructField("user_id",StringType(),False),
    StructField("interests",ArrayType(StringType()),True)
])

In [11]:
df_interests = spark.createDataFrame(interest_data, schema=interest_schema)
df_interests.printSchema()
df_interests.show(truncate=False)

root
 |-- user_id: string (nullable = false)
 |-- interests: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------+--------------------------------+
|user_id|interests                       |
+-------+--------------------------------+
|U001   |[AI, ML, Cloud]                 |
|U002   |[Testing, Automation]           |
|U003   |[Data Engineering, Spark, Kafka]|
|U004   |[UI/UX]                         |
+-------+--------------------------------+



In [13]:
from pyspark.sql.functions import explode
df_interests.select(
    "user_id",
    explode("interests").alias("interest")

).show()

+-------+----------------+
|user_id|        interest|
+-------+----------------+
|   U001|              AI|
|   U001|              ML|
|   U001|           Cloud|
|   U002|         Testing|
|   U002|      Automation|
|   U003|Data Engineering|
|   U003|           Spark|
|   U003|           Kafka|
|   U004|           UI/UX|
+-------+----------------+



In [14]:
from pyspark.sql.types import MapType

In [15]:
device_data = [
    ("U001",{"mobile":120,"laptop":300}),
    ("U002",{"tablet":80}),
    ("U003",{"mobile":200,"desktop":400}),
    ("U004",{"laptop":250})
]

In [16]:
device_schema = StructType([
    StructField("user_id",StringType(),False),
    StructField("device_usage",MapType(StringType(),IntegerType()),True)
])

In [17]:
df_devices = spark.createDataFrame(device_data, schema=device_schema)
df_devices.printSchema()
df_devices.show(truncate=False)

root
 |-- user_id: string (nullable = false)
 |-- device_usage: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)

+-------+-------------------------------+
|user_id|device_usage                   |
+-------+-------------------------------+
|U001   |{mobile -> 120, laptop -> 300} |
|U002   |{tablet -> 80}                 |
|U003   |{mobile -> 200, desktop -> 400}|
|U004   |{laptop -> 250}                |
+-------+-------------------------------+



In [18]:
nested_data = [
    ("U001",("Hyderabad","Telangana",500081)),
    ("U002",("Delhi","Delhi",110001)),
    ("U003",("Bangalore","Karnataka",560001))
]

In [19]:
address_schema = StructType([
    StructField("city",StringType(),True),
    StructField("state",StringType(),True),
    StructField("pincode",IntegerType(),True)
])

profile_schema = StructType([
    StructField("user_id",StringType(),False),
    StructField("address",address_schema,True)
])

In [20]:
df_profiles = spark.createDataFrame(nested_data, schema=profile_schema)
df_profiles.printSchema()
df_profiles.show(truncate=False)

root
 |-- user_id: string (nullable = false)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- pincode: integer (nullable = true)

+-------+------------------------------+
|user_id|address                       |
+-------+------------------------------+
|U001   |{Hyderabad, Telangana, 500081}|
|U002   |{Delhi, Delhi, 110001}        |
|U003   |{Bangalore, Karnataka, 560001}|
+-------+------------------------------+



In [21]:
df_profiles.select(
    "user_id",
    "address.city",
    "address.state"
).show()

+-------+---------+---------+
|user_id|     city|    state|
+-------+---------+---------+
|   U001|Hyderabad|Telangana|
|   U002|    Delhi|    Delhi|
|   U003|Bangalore|Karnataka|
+-------+---------+---------+



#Type casting

In [22]:
from pyspark.sql.functions import col
df_users.withColumn(
    "salary_int",
    col("salary").cast("int")
)

DataFrame[user_id: string, name: string, age: int, city: string, salary: bigint, salary_int: int]

#Convert to Date


In [ ]:
from pyspark.sql.functions import to_date

df_orders.withColumn(
    "order_date",
    to_date("order_date","yyyy-MM-dd")
)